In [ ]:
pip install imagecodecs

     |████████████████████████████████| 30.0MB 99kB/s 


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from PIL import Image,ImageOps
from google.colab import files
import os
from cv2 import cv2
import imagecodecs
import random

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# --------------------------------------MODEL 1 -------------------------------------
def create_model(xdim,ydim,model_num):
  channel = 1
  if model_num == 1:
      lenet_5_model = keras.models.Sequential([
      keras.layers.Conv2D(6, kernel_size=5, strides=1,  activation='tanh', input_shape=(xdim,ydim,channel), padding='same'), # input_shape is only the image shape. 
      keras.layers.AveragePooling2D(), #S2
      keras.layers.Conv2D(16, kernel_size=5, strides=1, activation='tanh', padding='valid'), #C3
      keras.layers.AveragePooling2D(), #S4
      keras.layers.Flatten(), #Flatten
      keras.layers.Dense(120, activation='tanh'), #C5
      keras.layers.Dense(84, activation='tanh'), #F6
      keras.layers.Dense(3, activation='softmax') #Output layer
      ])
      lenet_5_model.compile( optimizer= 'adam' , loss=keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])
      return lenet_5_model
  else:
    # ---------------------- AlexNet --------------
    model = keras.models.Sequential([
    keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(xdim,ydim,channel)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(3, activation='softmax')
])
    model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.optimizers.SGD(lr=0.001), metrics=['accuracy'])
    model.summary()
    return model
  

In [ ]:
# ----------------------------MODEL 2 ----------------------------------------------
import tensorflow as tf  # deep learning library. Tensors are just multi-dimensional arrays
model = tf.keras.models.Sequential()  # a basic feed-forward model
model.add(tf.keras.layers.Flatten())  # takes our 28x28 and makes it 1x784
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))  # a simple fully-connected layer, 128 units, relu activation
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))  # a simple fully-connected layer, 128 units, relu activation
model.add(tf.keras.layers.Dense(3, activation=tf.nn.softmax))  # our output layer. 10 units for 10 classes. Softmax for probability distribution
model.compile(optimizer='adam',  # Good default optimizer to start with
              loss='sparse_categorical_crossentropy',  # how will we calculate our "error." Neural network aims to minimize loss.
              metrics=['accuracy'])  # what to track

In [ ]:
def train_my_model(model_name,datasource,xdim):

  drive = datasource
  n = 0
  m = 0
  channel = 1
  ydim = xdim 
  counter = 0
  train_list = list()
  y = list()
  train_num = 0
  test_num = 0
  train_model = True
  total_trained = 0
  files = list()
  for filename in os.listdir(drive):
    files.append(filename)
  y_label_array = tf.keras.utils.to_categorical([0,1,2],3)
  #print(y_label_array)
  files_rand = random.sample(files, len(files))
  
  im_numx = 0
  im_num = 0
  for filename in files_rand:
      total_trained = total_trained +1
      #print("FILE NUMBERS : ",total_trained)
      filename = drive+"/"+filename    
      if "jxr" in filename:
        #print(filename)
        data = open(filename, 'rb').read()
        image = imagecodecs.jxr_decode(data)
        image = Image.fromarray(image).convert('RGB')
      elif "jpeg" in filename or "jp2" in filename:
          image = Image.open(filename)
      else:
        continue
      gray_image = ImageOps.grayscale(image)
      #gray_image = image
      imgwidth, imgheight = image.size
      if "jpeg" in filename:
        #print("JPEG")
        #y_label = [y_label_array[0]]
        y_label = np.array(0)
      elif "jp2" in filename:
        #print("JPEG 2000")
        #y_label = [y_label_array[1]]
        y_label = np.array(1)
      elif "jxr" in filename:
        #y_label = [y_label_array[2]]
        y_label = np.array(2)

      for i in range(0,imgheight,xdim):
          for j in range(0,imgwidth,ydim):
              im_num = im_num + 1 
              if im_num == 10:
                im_num = 0
                break
              box = (j, i, j+xdim, i+ydim)
              a = gray_image.crop(box)
              a = np.array(a)/255
              a = a.reshape(xdim,ydim,channel)
              train_list.append(a)
              y.append(y_label)
          if im_num == 0:
            break
  y = np.array(y)
  image_number = len(train_list)
  print("IMAGE NUMBER",image_number,"WINDOW SIZE",xdim)
  y = y.reshape(image_number,1)
  train_list = np.array(train_list)
  train_list = train_list.reshape(image_number,xdim,ydim,channel)
  model_name.fit(train_list,y,validation_split = 0.1,epochs=7)          
  #model_name.fit(train_list, y, epochs=10, batch_size=2,verbose=0)
  model_name.save("/content/gdrive/MyDrive/Datas/same ssim/09/Train/my_model")


In [ ]:
# ------------------------------------ EVALUATE --------------------------------
def evaluate_model(model,data_source,xdim):    
  ydim = xdim
  channel = 1
  folder_name = data_source
  temp_dic = {}
  temp_dic["JPEG"] = {}
  temp_dic["JPEG2000"] = {}
  temp_dic["JPEGXR"] = {}

  temp_dic["JPEG"]["true"] = 0
  temp_dic["JPEG"]["false"] = 0
  temp_dic["JPEG"]["image"] = 0
  
  temp_dic["JPEG2000"]["true"] = 0
  temp_dic["JPEG2000"]["false"] = 0
  temp_dic["JPEG2000"]["image"] = 0

  temp_dic["JPEGXR"]["true"] = 0
  temp_dic["JPEGXR"]["false"] = 0
  temp_dic["JPEGXR"]["image"] = 0
  im_num = 0
  for filename in os.listdir(folder_name):
    print("FILE NAME : ",filename)
    filename = folder_name +"/" + filename     
    if "jxr" in filename:
        #print(filename)
        data = open(filename, 'rb').read()
        image = imagecodecs.jxr_decode(data)
        image = Image.fromarray(image).convert('RGB')
    elif "jpeg" in filename or "jp2" in filename:
          image = Image.open(filename)
    else:
      continue
          
          
    gray_image = ImageOps.grayscale(image)
    #gray_image = image
    imgwidth, imgheight = image.size
    y = list()
    if "jpeg" in filename:
      y_label = np.array(0)
    elif "jp2" in filename:
      y_label = np.array(1)
    else:
      y_label = np.array(2)
    for i in range(0,imgheight,xdim):
      for j in range(0,imgwidth,ydim):
          box = (j, i, j+xdim, i+ydim)
          a = gray_image.crop(box)
          a = np.array(a)/255
          a = a.reshape(1,xdim,ydim,channel)
          result = model.predict(a)
          maximum_val = np.amax(result)
          index = np.where(result == maximum_val)
          index = index[1]
          if "jxr" in filename:
            temp_dic["JPEGXR"]["image"] = temp_dic["JPEGXR"]["image"] + 1
            if index == 2:
              temp_dic["JPEGXR"]["true"] = temp_dic["JPEGXR"]["true"] + 1
            else:
              temp_dic["JPEGXR"]["false"] = temp_dic["JPEGXR"]["false"] + 1
          elif "jpeg" in filename:
            temp_dic["JPEG"]["image"] = temp_dic["JPEG"]["image"] + 1
            if index == 0:
              temp_dic["JPEG"]["true"] = temp_dic["JPEG"]["true"] + 1
            else:
              temp_dic["JPEG"]["false"] = temp_dic["JPEG"]["false"] + 1
          elif "jp2" in filename:
            temp_dic["JPEG2000"]["image"] = temp_dic["JPEG2000"]["image"] + 1
            if index == 1:
              temp_dic["JPEG2000"]["true"] = temp_dic["JPEG2000"]["true"] + 1
            else:
              temp_dic["JPEG2000"]["false"] = temp_dic["JPEG2000"]["false"] + 1
          im_num = im_num + 1
          if im_num == 10:
            im_num = 0
            break
      if im_num == 0:
          break




          #result = model.evaluate(a,y_label)
  print("JPEG CORRECT : ",temp_dic["JPEG"]["true"])
  print("JPEG FALSE : ",temp_dic["JPEG"]["false"])
  print("JPEG IMAGE AMOUNT : ",temp_dic["JPEG"]["image"])


  print("JPEG 2000 CORRECT : ",temp_dic["JPEG2000"]["true"])
  print("JPEG 2000 FALSE : ",temp_dic["JPEG2000"]["false"])
  print("JPEG 2000 IMAGE AMOUNT : ",temp_dic["JPEG2000"]["image"])

  print("JPEGXR CORRECT : ",temp_dic["JPEGXR"]["true"])
  print("JPEGXR FALSE : ",temp_dic["JPEGXR"]["false"])
  print("JPEGXR IMAGE AMOUNT : ",temp_dic["JPEGXR"]["image"])



In [ ]:
window_size = [80,100,120,150,180,200,220,240,280]
train_source = "/content/gdrive/MyDrive/Datas/same ssim/08/train"
test_source = "/content/gdrive/MyDrive/Datas/same ssim/08/test"
for window in window_size:
  model = create_model(window,window,1)
  train_my_model(model,train_source,window)
  evaluate_model(model,test_source,window)

IMAGE NUMBER 1899 WINDOW SIZE 80
Epoch 1/7
54/54 [==============================] - 8s 144ms/step - loss: 1.1019 - accuracy: 0.5044 - val_loss: 0.9342 - val_accuracy: 0.7158
Epoch 2/7
54/54 [==============================] - 8s 140ms/step - loss: 0.9825 - accuracy: 0.5524 - val_loss: 0.8760 - val_accuracy: 0.6947
Epoch 3/7
54/54 [==============================] - 8s 139ms/step - loss: 0.9840 - accuracy: 0.5500 - val_loss: 0.9447 - val_accuracy: 0.6947
Epoch 4/7
54/54 [==============================] - 7s 139ms/step - loss: 0.9884 - accuracy: 0.5524 - val_loss: 0.8835 - val_accuracy: 0.7053
Epoch 5/7
54/54 [==============================] - 7s 139ms/step - loss: 0.9730 - accuracy: 0.5512 - val_loss: 0.8941 - val_accuracy: 0.7158
Epoch 6/7
54/54 [==============================] - 7s 138ms/step - loss: 0.9748 - accuracy: 0.5524 - val_loss: 0.8596 - val_accuracy: 0.7105
Epoch 7/7
54/54 [==============================] - 7s 139ms/step - loss: 0.9725 - accuracy: 0.5535 - val_loss: 0.8913 - v

In [ ]:
lenet_5_model.save("/content/gdrive/MyDrive/Datas/same ssim/09/Train/my_model")

INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/Datas/same ssim/09/Train/my_model/assets
